In [1]:
%gui qt6
import numpy as np
import pandas as pd
import os
import usabilities as us
import formulas as form
import visualisation as visual
np.seterr(all='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [6]:
SiO2 = pd.read_csv(
    filepath_or_buffer=os.path.join('..', 'resources', 'SiO2_nk.csv'), 
    sep='\t',
    header=1,
    index_col=3).reset_index(drop=True)
SiO2.head()

,um,n,k
0,0.250,1.513525,0.0
1,0.255,1.510877,0.0
2,0.260,1.508413,0.0
3,0.265,1.506117,0.0
4,0.270,1.503972,0.0


In [8]:
SiO2_params, SiO2_loss = us.fit_OC(
    x=SiO2['um'],
    y=SiO2['n'],
    functions=[form.cauchy],
    functions_parameters_number=[3],
    bounds=[(1.3,1.55), (1e-5,0.1), (1e-7,0.1)]
)
SiO2_params, SiO2_loss

(array([1.45260948e+00, 3.70987731e-03, 1.87993270e-06]),
 0.00040252273881933155)

In [9]:
visual.nk_plot(
    wavelength=SiO2['um'],
    n=SiO2['n']
)

In [10]:
visual.compare_results(
    x_space=SiO2['um'],
    y_pred=form.cauchy(
        parameters=SiO2_params,
        wavelength=SiO2['um']),
    y_true=SiO2['n']
)

In [14]:
Si = pd.read_csv(
    filepath_or_buffer=os.path.join('..', 'resources', 'Si_e.csv'), 
    sep='\t', 
    header=1,
    index_col=3).reset_index(drop=True)
Si.head()

,eV,e1,e2
0,4.960000,-10.647780,12.186640
1,4.862745,-11.955365,13.009671
2,4.769231,-13.629112,14.336851
3,4.679246,-15.588576,16.423994
4,4.592593,-17.702778,19.582952


In [12]:
Si_params, Si_loss = us.fit_OC(
    x=Si['eV'],
    y=Si[['e1', 'e2']],
    functions=[form.TL for _ in range(3)],
    functions_parameters_number=[5 for _ in range(3)],
    bounds=[bound for _ in range(3) for bound in [(0.001, 500), (3,5), (0.001,3), (2,4), (0.5,2)]]
)

In [20]:
Si_params, Si_loss

(array([4.98219597e+02, 3.37789769e+00, 2.37070598e-01, 3.01729446e+00,
        5.05605205e-01, 2.08894913e+02, 3.76978490e+00, 9.89859236e-01,
        2.51636090e+00, 6.08195688e-01, 6.15050994e+01, 4.23609142e+00,
        3.79184729e-01, 2.50515374e+00, 5.32112890e-01]),
 0.28715569547592973)

In [15]:
visual.e_plot(
    ph_energy=Si['eV'],
    e1=Si['e1'],
    e2=Si['e2']
)

In [23]:
visual.compare_results(
    x_space=Si['eV'],
    y_pred=us.multi_model(
        Si_params,
        Si['eV'],
        [form.TL for _ in range(3)],
        [5 for _ in range(3)]
    ),
    y_true=Si[['e1', 'e2']]
)

In [8]:
TiO2 = pd.read_csv(os.path.join('..', 'resources', 'TiO2.csv'), sep='\t', index_col=3).reset_index(drop=True)
TiO2.head()

,eV,e1,e2
0,4.95,3.959499,5.979951
1,4.90,4.168708,6.052847
2,4.85,4.392638,6.116233
3,4.80,4.631594,6.168044
4,4.75,4.885654,6.205946


In [9]:
TiO2_params, TiO2_loss = us.fit_OC(
    x=TiO2['eV'],
    y=TiO2[['e1', 'e2']],
    functions=[form.TL],
    functions_parameters_number=[5],
    bounds=[(0.001,1000), (4,7), (0.001,5), (3.2,3.5), (0,10)]
)
TiO2_params, TiO2_loss

(array([261.35026378,   4.        ,   1.81904225,   3.39251031,
          2.15898972]),
 0.0361125643838852)

In [10]:
visual.e_plot(
    ph_energy=TiO2['eV'],
    e1=TiO2['e1'],
    e2=TiO2['e2']
)

In [11]:
visual.compare_results(
    x_space=TiO2['eV'],
    y_pred=us.multi_model(
        TiO2_params,
        TiO2['eV'],
        [form.TL for _ in range(1)],
        [5 for _ in range(1)]
    ),
    y_true=TiO2[['e1', 'e2']]
)

In [2]:
H2O = pd.read_csv(os.path.join('..', 'resources', 'H2O.csv'), sep='\t', index_col=3).reset_index(drop=True)
H2O.head()

,um,n,k
0,0.250505,1.396272,0.0
1,0.253061,1.394556,0.0
2,0.255670,1.392870,0.0
3,0.258333,1.391213,0.0
4,0.261053,1.389585,0.0


In [3]:
def Sellmeier(parameters, wavelength):
    n = np.sqrt(1 + (parameters[0]*wavelength**2)/(wavelength**2 - parameters[1]) 
                + (parameters[2]*wavelength**2)/(wavelength**2 - parameters[3]))
    return n

In [4]:
H2O_params, H2O_loss = us.fit_OC(
    x=H2O['um'],
    y=H2O['n'],
    functions=[Sellmeier],
    functions_parameters_number=[4],
    bounds=[(0.001,10), (0.001,10), (0.001,10), (0.001,10)]
)
H2O_params, H2O_loss

(array([0.18172696, 9.966888  , 0.75667304, 0.01285388]),
 0.00012290401829242442)

In [5]:
visual.nk_plot(
    wavelength=H2O['um'],
    n=H2O['n']
)

In [6]:
visual.compare_results(
    x_space=H2O['um'],
    y_pred=Sellmeier(
        parameters=H2O_params,
        wavelength=H2O['um']),
    y_true=H2O['n']
)